# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
from holoviews import opts
import warnings
warnings.filterwarnings("ignore")


# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data1/cities1.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,139.922222,91,25,4.29,CL,1680940476
1,1,santa elena,-2.2333,-80.8500,148.122222,80,82,0.64,EC,1680940236
2,2,langres,47.8333,5.3333,137.777778,84,7,2.96,FR,1680940477
3,3,shestakovo,56.4915,103.9600,130.650000,56,100,4.35,RU,1680940477
4,4,new norfolk,-42.7826,147.0587,139.927778,67,17,1.79,AU,1680940477


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=2,
    color="City",
    hover_cols=["Lat", "Lng", "City", "Humidity"]
)

#map_plot1.opts(width=800, height=600)

# Display the map


map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_conditions = (city_data_df["Max Temp"] >90 ) & (city_data_df["Wind Speed"] < 2.5) & (city_data_df["Cloudiness"] == 0)
ideal_cities_df = city_data_df[ideal_conditions]

#filtered_cities_df = city_data_df[(city_data_df["Max Temp"] > max_temp) & (city_data_df["Cloudiness"] == cloudiness)]


# Drop any rows with null values
# YOUR CODE HERE
ideal_cities_df.dropna()


# Display sample data
# YOUR CODE HERE
ideal_cities_df.head()





,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,sheridan,44.8333,-106.9173,136.538889,75,0,1.54,US,1680940485
77,77,okhotsk,59.3833,143.3000,129.605556,74,0,1.31,RU,1680940505
83,83,durango,24.8333,-104.8333,139.844444,41,0,1.68,MX,1680940387
109,109,lompoc,34.6391,-120.4579,140.861111,84,0,1.34,US,1680940519
114,114,perpignan,42.7500,2.7500,141.938889,62,0,0.51,FR,1680940521


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,sheridan,US,44.8333,-106.9173,75,
77,okhotsk,RU,59.3833,143.3000,74,
83,durango,MX,24.8333,-104.8333,41,
109,lompoc,US,34.6391,-120.4579,84,
114,perpignan,FR,42.7500,2.7500,62,
119,saint george,US,37.1041,-113.5841,45,
164,yuanping,CN,38.7153,112.7575,13,
189,roald,NO,62.5833,6.1333,65,
204,ocampo,MX,21.6500,-101.5000,62,
205,umm lajj,SA,25.0213,37.2685,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel

radius = 10000
limit=20
#filters = f"circle:{longitude},{latitude},{radius}"
#bias = f"proximity:{longitude},{latitude}"
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat =hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{lng},{lat},{radius}"

    params["bias"] = f"proximity:{lng},{lat}"
        #params["categories"] = f"accomodation.hotel.{filter}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sheridan - nearest hotel: Comfort Inn & Suites
okhotsk - nearest hotel: No hotel found
durango - nearest hotel: No hotel found
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
perpignan - nearest hotel: Mas de l'Alzine
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
yuanping - nearest hotel: No hotel found
roald - nearest hotel: Vigra fjordhotell
ocampo - nearest hotel: No hotel found
umm lajj - nearest hotel: No hotel found
benemerito de las americas - nearest hotel: hotel
ixtapa - nearest hotel: Ma. Cristina
williamsport - nearest hotel: TownePlace Suites Williamsport
zunheboto - nearest hotel: No hotel found
garden city - nearest hotel: Quality Inn & Suites
nome - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
coahuayana - nearest hotel: No hotel found
dinanagar - nearest hotel: No hotel found
chiredzi - nearest hotel: Nesbitt Arms Hotel
aksu - nearest hotel: Akesu Tianyuan International Hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
25,sheridan,US,44.8333,-106.9173,75,Comfort Inn & Suites
77,okhotsk,RU,59.3833,143.3000,74,No hotel found
83,durango,MX,24.8333,-104.8333,41,No hotel found
109,lompoc,US,34.6391,-120.4579,84,Embassy Suites by Hilton Lompoc Central Coast
114,perpignan,FR,42.7500,2.7500,62,Mas de l'Alzine
119,saint george,US,37.1041,-113.5841,45,"The Advenire, an Autograph Collection Hotel"
164,yuanping,CN,38.7153,112.7575,13,No hotel found
189,roald,NO,62.5833,6.1333,65,Vigra fjordhotell
204,ocampo,MX,21.6500,-101.5000,62,No hotel found
205,umm lajj,SA,25.0213,37.2685,34,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [87]:
%%capture --no-display

# Configure the map plot
hotel_map=hotel_df.hvplot.points("Lat","Lng",geo=True, tiles ="OSM", frame_width=800, frame_height=600,
                                 size="Humidity", scale=2, 
                                 color="City", 
                                 hover_cols = ["Lat","Lng","City","Humidity","Hotel Name","Country"]
            )

# Display the map
hotel_map 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City,Humidity,Hotel Name,Country)